In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint

In [46]:
file=pd.read_csv("time_series_covid19_confirmed_US.csv")
file.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2021,2023,2030,2048,2059,2074,2082,2103,2126,2141
1,84001003,US,USA,840,1003.0,Baldwin,f,US,30.727750,-87.722071,...,6443,6475,6615,6637,6658,6694,6712,6743,6768,6888
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,996,997,1012,1031,1033,1033,1042,1045,1055,1056
3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,801,811,825,828,840,843,850,856,861,866
4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,1873,1893,1911,1925,1932,1942,1972,1988,2009,2039


In [47]:
state_df=file.set_index("Province_State")
state_df

,UID,iso2,iso3,code3,FIPS,Admin2,Country_Region,Lat,Long_,Combined_Key,...,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20
Province_State,,,,,,,,,,,,,,,,,,,,,
Alabama,84001001,US,USA,840,1001.0,Autauga,US,32.539527,-86.644082,"Autauga, Alabama, US",...,2021,2023,2030,2048,2059,2074,2082,2103,2126,2141
f,84001003,US,USA,840,1003.0,Baldwin,US,30.727750,-87.722071,"Baldwin, Alabama, US",...,6443,6475,6615,6637,6658,6694,6712,6743,6768,6888
Alabama,84001005,US,USA,840,1005.0,Barbour,US,31.868263,-85.387129,"Barbour, Alabama, US",...,996,997,1012,1031,1033,1033,1042,1045,1055,1056
Alabama,84001007,US,USA,840,1007.0,Bibb,US,32.996421,-87.125115,"Bibb, Alabama, US",...,801,811,825,828,840,843,850,856,861,866
Alabama,84001009,US,USA,840,1009.0,Blount,US,33.982109,-86.567906,"Blount, Alabama, US",...,1873,1893,1911,1925,1932,1942,1972,1988,2009,2039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wyoming,84056039,US,USA,840,56039.0,Teton,US,43.935225,-110.589080,"Teton, Wyoming, US",...,715,727,740,743,747,756,761,772,779,796
Wyoming,84056041,US,USA,840,56041.0,Uinta,US,41.287818,-110.547578,"Uinta, Wyoming, US",...,407,410,425,428,431,442,451,462,469,479
Wyoming,84090056,US,USA,840,90056.0,Unassigned,US,0.000000,0.000000,"Unassigned, Wyoming, US",...,0,0,0,0,0,0,0,0,0,0


In [48]:
state_ca_tx=state_df.loc[["California","Texas"]]
state_ca_tx

,UID,iso2,iso3,code3,FIPS,Admin2,Country_Region,Lat,Long_,Combined_Key,...,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20
Province_State,,,,,,,,,,,,,,,,,,,,,
California,84006001,US,USA,840,6001.0,Alameda,US,37.646294,-121.892927,"Alameda, California, US",...,22932,23001,23133,23215,23312,23391,23471,23576,23636,23775
California,84006003,US,USA,840,6003.0,Alpine,US,38.596786,-119.822359,"Alpine, California, US",...,3,3,3,3,3,3,3,3,4,8
California,84006005,US,USA,840,6005.0,Amador,US,38.445831,-120.656960,"Amador, California, US",...,313,313,317,324,328,330,331,335,337,338
California,84006007,US,USA,840,6007.0,Butte,US,39.667278,-121.600525,"Butte, California, US",...,3026,3041,3046,3059,3078,3087,3095,3107,3134,3143
California,84006009,US,USA,840,6009.0,Calaveras,US,38.205371,-120.552913,"Calaveras, California, US",...,339,339,339,342,342,342,342,347,347,347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Texas,84048499,US,USA,840,48499.0,Wood,US,32.787224,-95.382364,"Wood, Texas, US",...,621,621,623,626,626,633,636,639,655,655
Texas,84048501,US,USA,840,48501.0,Yoakum,US,33.173202,-102.827643,"Yoakum, Texas, US",...,289,289,289,289,289,289,289,300,304,319
Texas,84048503,US,USA,840,48503.0,Young,US,33.176597,-98.687909,"Young, Texas, US",...,497,497,507,535,535,535,559,578,591,605


In [49]:
total_by_state=state_ca_tx.groupby("Province_State").sum().drop_duplicates()
total_by_state=total_by_state.reset_index()
total_by_state[["Province_State","10/30/20"]]

,Province_State,10/30/20
0,California,928028
1,Texas,924447


In [50]:
state_ca_tx.columns

Index(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Country_Region',
       'Lat', 'Long_', 'Combined_Key',
       ...
       '10/21/20', '10/22/20', '10/23/20', '10/24/20', '10/25/20', '10/26/20',
       '10/27/20', '10/28/20', '10/29/20', '10/30/20'],
      dtype='object', length=293)

In [51]:
new_state_ca_tx=state_ca_tx.drop(columns=['UID','iso2','iso3','code3','FIPS','Country_Region','Lat','Long_','Combined_Key'])
new_state_ca_tx 

                                          
                                          

,Admin2,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20
Province_State,,,,,,,,,,,,,,,,,,,,,
California,Alameda,0,0,0,0,0,0,0,0,0,...,22932,23001,23133,23215,23312,23391,23471,23576,23636,23775
California,Alpine,0,0,0,0,0,0,0,0,0,...,3,3,3,3,3,3,3,3,4,8
California,Amador,0,0,0,0,0,0,0,0,0,...,313,313,317,324,328,330,331,335,337,338
California,Butte,0,0,0,0,0,0,0,0,0,...,3026,3041,3046,3059,3078,3087,3095,3107,3134,3143
California,Calaveras,0,0,0,0,0,0,0,0,0,...,339,339,339,342,342,342,342,347,347,347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Texas,Wood,0,0,0,0,0,0,0,0,0,...,621,621,623,626,626,633,636,639,655,655
Texas,Yoakum,0,0,0,0,0,0,0,0,0,...,289,289,289,289,289,289,289,300,304,319
Texas,Young,0,0,0,0,0,0,0,0,0,...,497,497,507,535,535,535,559,578,591,605


In [ ]:
total_by_state.columns
df_new=total_by_state.drop(columns=['UID','code3','FIPS','Lat','Long_'])
df_new

In [53]:
transpose_df=df_new.T
transpose_df
new_t=transpose_df.reset_index()
new_t

,index,0,1
0,Province_State,California,Texas
1,1/22/20,0,0
2,1/23/20,0,0
3,1/24/20,0,0
4,1/25/20,0,0
...,...,...,...
279,10/26/20,910438,898302
280,10/27/20,914888,904769
281,10/28/20,919276,911835
282,10/29/20,923648,918721


In [54]:
new_t.dtypes
new_t=new_t.rename(columns=new_t.iloc[0])
new_t

,Province_State,California,Texas
0,Province_State,California,Texas
1,1/22/20,0,0
2,1/23/20,0,0
3,1/24/20,0,0
4,1/25/20,0,0
...,...,...,...
279,10/26/20,910438,898302
280,10/27/20,914888,904769
281,10/28/20,919276,911835
282,10/29/20,923648,918721


In [55]:

clean_data=new_t.drop(0)
clean_data


,Province_State,California,Texas
1,1/22/20,0,0
2,1/23/20,0,0
3,1/24/20,0,0
4,1/25/20,0,0
5,1/26/20,2,0
...,...,...,...
279,10/26/20,910438,898302
280,10/27/20,914888,904769
281,10/28/20,919276,911835
282,10/29/20,923648,918721


In [71]:
difference=clean_data.set_index("Province_State").diff()
difference.tail(40)

,California,Texas
Province_State,,
9/21/20,3928,21771
9/22/20,2969,6585
9/23/20,3371,1550
9/24/20,3080,4453
9/25/20,4144,3702
9/26/20,3791,3243
9/27/20,2439,4445
9/28/20,2821,2442
9/29/20,2943,6066
